**Author**: [Michael Gschwind](https://github.com/mikekgfb)

This tutorial introduces Better Transformer (BT) as part of the PyTorch
1.12 release. In this tutorial, we show how to use Better Transformer
for production inference with torchtext. Better Transformer is a
production ready fastpath to accelerate deployment of Transformer models
with high performance on CPU and GPU. The fastpath feature works
transparently for models based either directly on PyTorch core
`nn.module` or with torchtext.

Models which can be accelerated by Better Transformer fastpath execution
are those using the following PyTorch core `torch.nn.module` classes
`TransformerEncoder`, `TransformerEncoderLayer`, and
`MultiHeadAttention`. In addition, torchtext has been updated to use the
core library modules to benefit from fastpath acceleration. (Additional
modules may be enabled with fastpath execution in the future.)

Better Transformer offers two types of acceleration:

-   Native multihead attention (MHA) implementation for CPU and GPU to
    improve overall execution efficiency.
-   Exploiting sparsity in NLP inference. Because of variable input
    lengths, input tokens may contain a large number of padding tokens
    for which processing may be skipped, delivering significant
    speedups.

Fastpath execution is subject to some criteria. Most importantly, the
model must be executed in inference mode and operate on input tensors
that do not collect gradient tape information (e.g., running with
torch.no_grad).

To follow this example in Google Colab, [click
here](https://colab.research.google.com/drive/1KZnMJYhYkOMYtNIX5S3AGIYnjyG0AojN?usp=sharing).



# Better Transformer Features in This Tutorial

-   Load pretrained models (created before PyTorch version 1.12 without
    Better Transformer)
-   Run and benchmark inference on CPU with and without BT fastpath
    (native MHA only)
-   Run and benchmark inference on (configurable) DEVICE with and
    without BT fastpath (native MHA only)
-   Enable sparsity support
-   Run and benchmark inference on (configurable) DEVICE with and
    without BT fastpath (native MHA + sparsity)

# Additional Information

Additional information about Better Transformer may be found in the
PyTorch.Org blog [A Better Transformer for Fast Transformer
Inference](https://pytorch.org/blog/a-better-transformer-for-fast-transformer-encoder-inference//).

1.  Setup

1.1 Load pretrained models

We download the XLM-R model from the predefined torchtext models by
following the instructions in
[torchtext.models](https://pytorch.org/text/main/models.html). We also
set the DEVICE to execute on-accelerator tests. (Enable GPU execution
for your environment as appropriate.)



In [5]:
import torch
import torch.nn as nn

print(f"torch version: {torch.__version__}")

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"torch cuda available: {torch.cuda.is_available()}")

import torch, torchtext
from torchtext.models import RobertaClassificationHead
from torchtext.functional import to_tensor
xlmr_large = torchtext.models.XLMR_LARGE_ENCODER 
classifier_head = torchtext.models.RobertaClassificationHead(num_classes=2, input_dim = 1024)
model = xlmr_large.get_model(head=classifier_head)
transform = xlmr_large.transform() # tokenizer

torch version: 2.1.2+cu121
torch cuda available: True


Downloading: "https://download.pytorch.org/models/text/xlmr.large.encoder.pt" to C:\Users\LihengLuo/.cache\torch\hub\checkpoints\xlmr.large.encoder.pt
100%|██████████| 2.08G/2.08G [01:12<00:00, 30.7MB/s] 
100%|██████████| 5.07M/5.07M [00:01<00:00, 3.23MB/s]
Downloading: "https://download.pytorch.org/models/text/xlmr.vocab.pt" to C:\Users\LihengLuo/.cache\torch\hub\checkpoints\xlmr.vocab.pt
100%|██████████| 4.85M/4.85M [00:01<00:00, 4.05MB/s]


In [6]:
small_input_batch = ["Hello world", "How are you!"]
big_input_batch = [
    "Hello world",  
    "How are you!",
    """`Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.`

It was in July, 1805, and the speaker was the well-known Anna
Pavlovna Scherer, maid of honor and favorite of the Empress Marya
Fedorovna. With these words she greeted Prince Vasili Kuragin, a man
of high rank and importance, who was the first to arrive at her
reception. Anna Pavlovna had had a cough for some days. She was, as
she said, suffering from la grippe; grippe being then a new word in
St. Petersburg, used only by the elite.""",
]

1.2 Dataset Setup

We set up two types of inputs: a small input batch and a big input batch
with sparsity.



Next, we select either the small or large input batch, preprocess the
inputs and test the model.

In [8]:
input_batch = big_input_batch

model_input = to_tensor(transform(input_batch), padding_value=1)
output = model(model_input)
output.shape

torch.Size([3, 2])

Finally, we set the benchmark iteration count:


In [9]:
ITERATIONS = 10

2.  Execution

2.1 Run and benchmark inference on CPU with and without BT fastpath
(native MHA only)

We run the model on CPU, and collect profile information:

-   The first run uses traditional ("slow path") execution.
-   The second run enables BT fastpath execution by putting the model in
    inference mode using <span class="title-ref">model.eval()</span> and
    disables gradient collection with <span
    class="title-ref">torch.no_grad()</span>.

You can see an improvement (whose magnitude will depend on the CPU
model) when the model is executing on CPU. Notice that the fastpath
profile shows most of the execution time in the native <span
class="title-ref">TransformerEncoderLayer</span> implementation <span
class="title-ref">aten::\_transformer_encoder_layer_fwd</span>.

In [10]:
print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
    for i in range(ITERATIONS):
        output = model(model_input)
print(prof)

model.eval() # disable dropout

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
    with torch.no_grad(): # disable gradient calculation
        for i in range(ITERATIONS):
            output = model(model_input)
print(prof)

slow path:
--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                    aten::eq         0.00%     137.000us         0.00%     137.000us     137.000us             1  
                             aten::embedding         0.00%     320.000us         0.01%       1.035ms       1.035ms             1  
                               aten::reshape         0.00%       3.000us         0.00%       8.000us       8.000us             1  
                                  aten::view         0.00%       5.000us         0.00%       5.000us       5.000us             1  
                          aten::index_select         0.00%     505.000us

c:\Users\LihengLuo\.conda\envs\gnn_cuda121\Lib\site-packages\torch\nn\modules\transformer.py:380: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ..\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                   aten::eq         0.00%      51.000us         0.00%      51.000us      51.000us             1  
                            aten::embedding         0.00%      22.000us         0.03%       1.796ms       1.796ms             1  
                              aten::reshape         0.00%       4.000us         0.00%       9.000us       9.000us             1  
                                 aten::view         0.00%       5.000us         0.00%       5.000us       5.000us             1  
                         aten::index_select         0.03%       1.588ms         0.03%     

2.2 Run and benchmark inference on (configurable) DEVICE with and
without BT fastpath (native MHA only)

We check the BT sparsity setting:

In [11]:
model.encoder.transformer.layers.enable_nested_tensor

True

We disable the BT sparsity:


In [12]:
model.encoder.transformer.layers.enable_nested_tensor = False

We run the model on DEVICE, and collect profile information for native
MHA execution on DEVICE:

-   The first run uses traditional ("slow path") execution.
-   The second run enables BT fastpath execution by putting the model in
    inference mode using <span class="title-ref">model.eval()</span> and
    disables gradient collection with <span
    class="title-ref">torch.no_grad()</span>.

When executing on a GPU, you should see a significant speedup, in
particular for the small input batch setting:

In [13]:
model.to(DEVICE)
model_input = model_input.to(DEVICE)

print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    for i in range(ITERATIONS):
        output = model(model_input)
print(prof)

model.eval()

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    with torch.no_grad():
        for i in range(ITERATIONS):
            output = model(model_input)
print(prof)

slow path:
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                         aten::eq         1.74%      25.911ms         1.74%      25.911ms      25.911ms      25.837ms         1.47%      25.837ms      25.837ms             1  
                                  aten::embedding         0.28%       4.157ms        19.28%     287.538ms     287.538ms       3.226ms         0.18%     286.828ms     286.828ms             1  
                            

2.3 Run and benchmark inference on (configurable) DEVICE with and
without BT fastpath (native MHA + sparsity)

We enable sparsity support:

In [14]:
model.encoder.transformer.layers.enable_nested_tensor = True

We run the model on DEVICE, and collect profile information for native
MHA and sparsity support execution on DEVICE:

-   The first run uses traditional ("slow path") execution.
-   The second run enables BT fastpath execution by putting the model in
    inference mode using <span class="title-ref">model.eval()</span> and
    disables gradient collection with <span
    class="title-ref">torch.no_grad()</span>.

When executing on a GPU, you should see a significant speedup, in
particular for the large input batch setting which includes sparsity:

In [15]:
model.to(DEVICE)
model_input = model_input.to(DEVICE)

print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    for i in range(ITERATIONS):
        output = model(model_input)
print(prof)

model.eval()

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    with torch.no_grad():
        for i in range(ITERATIONS):
            output = model(model_input)
print(prof)

slow path:
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                         aten::eq         0.01%     136.000us         0.01%     136.000us     136.000us       9.000us         0.00%       9.000us       9.000us             1  
                                  aten::embedding         0.01%      87.000us         0.02%     219.000us     219.000us      70.000us         0.01%     230.000us     230.000us             1  
                            

# Summary

In this tutorial, we have introduced fast transformer inference with
Better Transformer fastpath execution in torchtext using PyTorch core
Better Transformer support for Transformer Encoder models. We have
demonstrated the use of Better Transformer with models trained prior to
the availability of BT fastpath execution. We have demonstrated and
benchmarked the use of both BT fastpath execution modes, native MHA
execution and BT sparsity acceleration.